In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../dataset/base_datos_2008.csv', nrows = 100000)
df

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,...,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2008,1,25,5,2204.0,1935,106.0,2257,XE,7676,...,4.0,17.0,0,NaN,0,0.0,0.0,0.0,0.0,129.0
99996,2008,1,26,6,1933.0,1935,2234.0,2257,XE,7676,...,4.0,16.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
99997,2008,1,27,7,2047.0,1935,2400.0,2257,XE,7676,...,6.0,18.0,0,NaN,0,23.0,0.0,0.0,0.0,40.0
99998,2008,1,28,1,1956.0,1935,2308.0,2257,XE,7676,...,5.0,25.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [4]:
# Detectar valores extremadamente pequeños o grandes - Outlayers
# trabajaremos con la columna ArrDelay y eliminaremos los datos faltantes de esta misma
x = df[ 'ArrDelay'].dropna()


In [6]:
# Cuartiles
# Medida estadistica que separan el 25% de los datos inferiores del 75% de los datos superiores ó 
# El 75% inferior del 25% superior de los datos ordenados

Q1 = np.percentile(x, 25)
# La mediana es el segundo cuartil. Q2
Q3 = np.percentile(x, 75)

# Rango intercuartílico
# Diferencia entre el tercer cuartil y el primer cuartil

rango_inter = Q3 - Q1

In [7]:
umbralsuperior = Q3 + 1.5 * rango_inter
umbralinferior = Q1 - 1.5 * rango_inter

In [8]:
umbralsuperior
# Cualquier valor que se encuentre sobre 38 va a ser un Outlayer

38.5

In [10]:
umbralinferior
# Cualquier valor que se encuentre debajo de -37 va a ser un outlayer

-37.5

In [12]:
# Comprobar cuantos casos hay sobre el umbral
np.mean(x > umbralsuperior)

0.08387201361729721

In [13]:
# Comprobar cuantos casos hay debajo del umbral
np.mean(x < umbralinferior)

0.0019048005025431114

Las medidas no son simetricas.
Lo que implica es que no estan igualmente distribuidas 

In [15]:
# Estudiar varias variables a la vez
from sklearn.covariance import EllipticEnvelope


In [16]:
# Modelo que va a seleccionar el 1% de datos que considere que estan muy alejados de nuestros datos mas centrados 
outliers = EllipticEnvelope( contamination = .01)

In [17]:
# Lista con grupo de variables de la base de datos
# Cuantitativas 
# Distribuidas gausianamente (son normales)
var_list = ['DepDelay', 'TaxiIn', 'TaxiOut', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

In [20]:
# Definimos x de nuevo
x = np.array( df.loc[:, var_list].dropna())

In [21]:
outliers.fit(x)

EllipticEnvelope(contamination=0.01)

In [22]:
pred = outliers.predict(x)

In [23]:
pred

array([1, 1, 1, ..., 1, 1, 1])

In [24]:
elips_outliers = np.where(pred == -1)[0]

In [25]:
# Valores fuera del centro de las variables 
elips_outliers

array([  132,   133,   202,   205,   818,   823,  1347,  1428,  1636,
        1681,  1843,  1958,  1968,  2255,  3008,  3010,  3897,  4259,
        4265,  4430,  4439,  4466,  4512,  4662,  4818,  4924,  4925,
        5138,  5140,  5160,  5162,  5229,  5230,  5331,  5555,  5577,
        5938,  6112,  6543,  7299,  7677,  7679,  7711,  8153,  8238,
        8279,  8482,  8483,  8567,  8618,  8792,  8806,  8819,  9508,
        9565, 10250, 10604, 10609, 10677, 10678, 10680, 10693, 10817,
       10818, 11022, 11099, 11100, 11102, 11105, 11529, 11635, 11897,
       11910, 11922, 12248, 12261, 12331, 13274, 13278, 13333, 13334,
       13335, 13336, 13337, 13338, 13722, 13723, 13726, 13727, 14257,
       14261, 14270, 14327, 14330, 14357, 14359, 14360, 14364, 14408,
       14425, 14427, 14491, 14492, 14493, 14515, 14583, 14585, 14647,
       14743, 14787, 14790, 14828, 14830, 14866, 14878, 14879, 14941,
       14942, 15033, 15037, 15090, 15096, 15104, 15106, 15107, 15134,
       15136, 15137,